In [1]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm

def fetch_f1_data():
    """Fetch complete F1 data from Ergast API (2020–2024)."""
    base_url = "http://ergast.com/api/f1/{year}/results.json?limit=100&offset={offset}"
    all_results = []
    
    for year in tqdm(range(2020, 2026), desc="Processing Seasons"):
        offset = 0
        while True:
            url = base_url.format(year=year, offset=offset)
            response = requests.get(url)
            data = response.json()
            races = data["MRData"]["RaceTable"]["Races"]
            if not races:
                break
                
            for race in races:
                for result in race.get("Results", []):
                    rec = {
                        "Season":    int(race["season"]),
                        "Round":     int(race["round"]),
                        "Circuit":   race["Circuit"]["circuitName"],
                        "Driver":    f"{result['Driver']['givenName']} {result['Driver']['familyName']}",
                        "Constructor": result["Constructor"]["name"],
                        "Grid":      int(result["grid"]),
                        "Position":  int(result["position"]) if result["position"].isdigit() else np.nan,
                        "Points":    float(result["points"]),
                        "Status":    result["status"],
                        "Laps":      int(result["laps"])
                    }
                    # fastest lap
                    if "FastestLap" in result and "Time" in result["FastestLap"]:
                        rec["FastestLapTime"]  = result["FastestLap"]["Time"]["time"]
                        rec["FastestLapSpeed"] = float(result["FastestLap"]["AverageSpeed"]["speed"])
                    all_results.append(rec)
            
            offset += 100
            if offset > int(data["MRData"]["total"]):
                break
                
    return pd.DataFrame(all_results)

# 1️⃣ Fetch raw data
df = fetch_f1_data()

# 2️⃣ Clean & convert types
df["Season"]  = pd.to_numeric(df["Season"], errors="coerce")
df["Round"]   = pd.to_numeric(df["Round"], errors="coerce")
df["Grid"]    = pd.to_numeric(df["Grid"], errors="coerce")
df["Position"]= pd.to_numeric(df["Position"], errors="coerce")
df["Points"]  = pd.to_numeric(df["Points"], errors="coerce")
df["Laps"]    = pd.to_numeric(df["Laps"], errors="coerce")
df["FastestLapSpeed"] = pd.to_numeric(df.get("FastestLapSpeed"), errors="coerce")

# convert FastestLapTime (e.g. "1:07.657") to seconds
def lap_time_to_sec(t):
    try:
        m, s = t.split(":")
        return int(m)*60 + float(s)
    except:
        return np.nan

df["FastestLapSec"] = df["FastestLapTime"].apply(lap_time_to_sec)

# drop any rows missing Season or Round
df.dropna(subset=["Season","Round"], inplace=True)

# 3️⃣ Feature engineering
df["Win"]    = (df["Position"] == 1).astype(int)
df["Podium"] = df["Position"].le(3).astype(int)

# 4️⃣ Aggregations

# Team wins per season
team_wins = (
    df.groupby(["Season","Constructor"])["Win"]
      .sum()
      .reset_index()
      .rename(columns={"Win":"Wins"})
)

# Driver stats overall
driver_stats = (
    df.groupby("Driver")
      .agg(
        TotalPoints=("Points","sum"),
        TotalWins=("Win","sum"),
        TotalPodiums=("Podium","sum"),
        Races=("Round","count"),
        AvgLapSpeed=("FastestLapSpeed","mean")
      )
      .reset_index()
)

# Driver performance by circuit
track_performance = (
    df.groupby(["Driver","Circuit"])
      .agg(
        AvgFinish=("Position","mean"),
        Count=("Round","count")
      )
      .reset_index()
)

# Optional lookups
constructors = df[["Constructor"]].drop_duplicates().reset_index(drop=True)
circuits     = df[["Circuit"]].drop_duplicates().reset_index(drop=True)

# 5️⃣ Export CSVs—UTF‑8 with BOM for Power BI compatibility
df.to_csv("f1_cleaned.csv",           index=False, encoding="utf-8-sig")
team_wins.to_csv("team_wins.csv",     index=False, encoding="utf-8-sig")
driver_stats.to_csv("driver_stats.csv", index=False, encoding="utf-8-sig")
track_performance.to_csv("track_performance.csv",
                         index=False, encoding="utf-8-sig")
constructors.to_csv("constructors_lookup.csv",
                    index=False, encoding="utf-8-sig")
circuits.to_csv("circuits_lookup.csv",
                index=False, encoding="utf-8-sig")

print("✅ All CSV files exported and ready to load into Power BI.")

Processing Seasons: 100%|████████████████████████████████████████████████████████████████| 6/6 [00:41<00:00,  6.83s/it]

✅ All CSV files exported and ready to load into Power BI.


In [2]:
print(df.head())

   Season  Round        Circuit           Driver Constructor  Grid  Position  \
0    2020      1  Red Bull Ring  Valtteri Bottas    Mercedes     1         1   
1    2020      1  Red Bull Ring  Charles Leclerc     Ferrari     7         2   
2    2020      1  Red Bull Ring     Lando Norris     McLaren     3         3   
3    2020      1  Red Bull Ring   Lewis Hamilton    Mercedes     5         4   
4    2020      1  Red Bull Ring     Carlos Sainz     McLaren     8         5   

   Points    Status  Laps FastestLapTime  FastestLapSpeed  FastestLapSec  Win  \
0    25.0  Finished    71       1:07.657          229.758         67.657    1   
1    18.0  Finished    71       1:07.901          228.933         67.901    0   
2    16.0  Finished    71       1:07.475          230.378         67.475    0   
3    12.0  Finished    71       1:07.712          229.572         67.712    0   
4    10.0  Finished    71       1:07.974          228.687         67.974    0   

   Podium  
0       1  
1       